# Baseline

In [1]:
import torch
from PIL import Image
import torchvision.transforms as T
from torch import nn
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
input_dir = "/kaggle/input/blood-vessel-segmentation/"
train_dir = input_dir + "train"


seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(seed)


print(device)

cuda


## Load data

**Datasets**

- kidney_1_dense	
    - images
    - labels
- kidney_1_voi  
    - images
    - labels
- kidney_2	
    - images
    - labels
- kidney_3_dense	
    - labels
- kidney_3_sparse
    - images
    - labels

### ¿Do I use one dataset or all of them? ¿How do I merge all the datasets?

I think i can merge the datasets by just merging the images and labels paths. But for now I will just use the `kidney_1_dense` dataset.

### Dataset

In [2]:
class KidneyDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, msk_paths, transform=None, target_transform=None):
        self.img_paths  = img_paths
        self.msk_paths  = msk_paths
        self.transform = transform
        self.target_transform = target_transform
        assert len(self.img_paths) == len(self.msk_paths)
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        msk_path = self.msk_paths[idx]

        img = Image.open(img_path)
        msk = Image.open(msk_path)

        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            msk = self.target_transform(msk)

        return img, msk

In [3]:
k1d_path = "kidney_1_dense"
h_resize, w_resize = 512, 512

folder_path = os.path.join(train_dir, k1d_path)
k1d_imgs_paths = sorted(os.path.join(folder_path, "images", file_name) for file_name in os.listdir(os.path.join(folder_path, "images")))
k1d_msks_paths = sorted(os.path.join(folder_path, "labels", file_name) for file_name in os.listdir(os.path.join(folder_path, "images")))

transform = T.Compose([
    T.ToTensor(),  
    T.ConvertImageDtype(torch.float32),
    T.Resize((h_resize, w_resize), antialias=True)
])

target_transform = T.Compose([
    T.PILToTensor(), 
    T.Resize((h_resize, w_resize), antialias=True)
])

k1d_ds = KidneyDataset(k1d_imgs_paths, k1d_msks_paths, transform=transform, target_transform=target_transform)

In [4]:
g = torch.Generator().manual_seed(42)

train_ds, eval_ds = torch.utils.data.random_split(
    k1d_ds,
    [0.8, 0.2],
    generator=g
)

bs = 32
num_workers = 2
train_dl = DataLoader(train_ds, batch_size=bs, num_workers=num_workers, shuffle=True)
eval_dl = DataLoader(eval_ds, batch_size=bs, num_workers=num_workers, shuffle=False)

In [5]:
len(train_ds), len(eval_ds)

(1824, 455)

## Define model

In [7]:
class SegmentationModel(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(SegmentationModel, self).__init__()

        # Define the encoder - a series of convolutional layers
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Define the decoder - a series of transposed convolutional layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, num_classes, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the model with the desired input channels and number of classes
model = SegmentationModel(in_channels=1, num_classes=1)
model.to(device)

# Use the model for a batch of input images
x, y = next(iter(train_dl))
x = x.to(device)
output = model(x)
print(output)
print(output.shape)

tensor([[[[-0.0177,  0.0010,  0.0009,  ..., -0.0031,  0.0007, -0.0039],
          [-0.0398, -0.0282, -0.0311,  ..., -0.0278, -0.0301, -0.0148],
          [-0.0323,  0.0054, -0.0092,  ...,  0.0008, -0.0063, -0.0043],
          ...,
          [-0.0378, -0.0238, -0.0265,  ..., -0.0311, -0.0258, -0.0125],
          [-0.0312,  0.0065, -0.0154,  ...,  0.0034, -0.0139, -0.0054],
          [-0.0202, -0.0193, -0.0081,  ..., -0.0210, -0.0087, -0.0184]]],


        [[[-0.0179,  0.0011,  0.0010,  ..., -0.0032,  0.0008, -0.0039],
          [-0.0400, -0.0282, -0.0316,  ..., -0.0278, -0.0302, -0.0148],
          [-0.0325,  0.0054, -0.0092,  ...,  0.0011, -0.0063, -0.0042],
          ...,
          [-0.0377, -0.0235, -0.0265,  ..., -0.0310, -0.0258, -0.0130],
          [-0.0311,  0.0061, -0.0152,  ...,  0.0029, -0.0140, -0.0055],
          [-0.0201, -0.0195, -0.0085,  ..., -0.0214, -0.0088, -0.0186]]],


        [[[-0.0178,  0.0006,  0.0007,  ..., -0.0027,  0.0003, -0.0037],
          [-0.0392, -0.027

## Define evaluation metric

In [ ]:
class IouMetric:
    def __init__(self, num_classes: int, int2str: dict, ignore_index: int = 255, prefix="train"):
        """
        Args:
            num_classes: number of classes
            int2str: dictionary mapping class index to class name
            ignore_index: index to ignore in the metric calculations
            prefix: prefix to use for logging
        """
        self.area_intersect = torch.zeros(num_classes)
        self.area_label = torch.zeros(num_classes)
        self.area_pred = torch.zeros(num_classes)
        self.num_classes = num_classes
        self.int2str = int2str
        self.ignore_index = ignore_index
        self.prefix = prefix

    def process(self, preds, labels):
        mask = labels != self.ignore_index
        preds = preds[mask]
        labels = labels[mask]

        # compute area of intersection, label and prediction
        intersect = preds[preds == labels]
        area_intersect = torch.histc(intersect.float(), bins=self.num_classes, min=0, max=self.num_classes - 1)
        area_label = torch.histc(labels.float(), bins=self.num_classes, min=0, max=self.num_classes - 1)
        area_pred = torch.histc(preds.float(), bins=self.num_classes, min=0, max=self.num_classes - 1)

        # update results
        self.area_intersect += area_intersect.cpu()
        self.area_label += area_label.cpu()
        self.area_pred += area_pred.cpu()

    def compute(self) -> dict:
        iou = self.area_intersect / (self.area_label + self.area_pred - self.area_intersect)
        if 0 <= self.ignore_index <= self.num_classes - 1:
            iou[self.ignore_index] = torch.nan
        mean_iou = torch.nanmean(iou)

        metrics = {
            f"{self.prefix}/iou_{self.int2str[idx]}": iou[idx].item()
            for idx in range(len(iou))
            if idx != self.ignore_index
        }
        metrics[f"{self.prefix}/mean_iou"] = mean_iou.item()

        return metrics

    def reset(self):
        self.area_intersect.zero_()
        self.area_label.zero_()
        self.area_pred.zero_()


## Define loss function


In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

## Train model

### Optimizer and scheduler

### Train method

### Evaluation method

## Make submisssion